<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 4: Создайте свой первый алгоритм обучения с глубоким подкреплением с помощью PyTorch: Reinforce. И проверьте его надежность 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail" width="50%"/>

В этом блокноте вы с нуля создадите свой первый алгоритм обучения с глубоким подкреплением: Reinforcement (также называемый градиентом политики Монте-Карло).

Усиление (Reinforce) - это *метод, основанный на политике*: алгоритм глубокого обучения с подкреплением, который пытается **оптимизировать политику напрямую, не используя функцию значения действия**.

Точнее, Усиление (Reinforce) - это *Метод градиента политики*, подкласс *методов, основанных на политике*, который направлен **на оптимизацию политики непосредственно путем оценки весов оптимальной политики с использованием градиентного подъема**.

Чтобы проверить его надежность, мы собираемся обучить его в двух разных простых средах:
- Cartpole-v1 (Столб тележки-v1)
- PixelcopterEnv (Пиксельный вертолет)

⬇️ Вот пример того, чего **вы достигнете в конце этом блокноте.** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments" width="50%"/>


### 🎮 Окружающая среда: 

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL-Библиотека: 

- Python
- PyTorch


 [GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Цели этого блокнота 🏆
В конце вы будете:
- Уметь **с нуля закодировать алгоритм усиления с использованием PyTorch.**
- Иметь возможность **протестировать надежность вашего агента, используя простые среды.**
- Иметь возможность **отправить своего обученного агента в центр** с хорошим видеоповтором и оценочным баллом 🔥 .

## Эта тетрадь взята из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

На этом бесплатном курсе вы:

- 📖 Изучите глубокое обучение с подкреплением в **теории и на практике**.
- 🧑 💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL и Sample Factory 2.0.
- 🤖 Обучите **агентов в уникальных условиях**

## Предпосылки 🏗️
Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 [Изучите градиенты политики, прочитав Unit 4](https://huggingface.co/deep-rl-course/unit4/introduction)

# Давайте запрограммируем алгоритм усиления с нуля 🔥


Чтобы пройти этот практический подход для сертификации, вам необходимо отправить свои обученные модели в Центр.

- Получите результат >= 350 для `Cart pole-v1`.
- Получите результат >= 5 для `Pixel Copter`.

Чтобы узнать свой результат, перейдите в таблицу лидеров и найдите свою модель, **the result = mean_reward - std_reward**. **Если вы не видите свою модель в таблице лидеров, перейдите в нижнюю часть страницы таблицы лидеров и нажмите на кнопку обновить**.

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

# Совет 💡
Лучше запустить этот colab в копии на вашем Google Диске, чтобы **в случае истечения времени ожидания** у вас все еще была сохраненная записная книжка на вашем Google Диске, и вам не нужно было заполнять все с нуля.

Для этого вы можете либо нажать `Ctrl + S` либо `File > Save a copy in Google Drive.`

## Запустить на GPU 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор GPU. Перейдите в `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="50%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="50%">

## Создание виртуального дисплея #

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1